<a href="https://colab.research.google.com/github/Roaa27/vh/blob/main/QAWT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Install and Import Libraries  
I installed Hugging Face Transformers, Datasets, and Evaluate libraries.  
These are used to load pre-trained models, manage datasets, and calculate evaluation metrics.

In [ ]:
!pip install transformers datasets evaluate

import pandas as pd
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from datasets import load_dataset
import evaluate

print("✅ Libraries installed and imported successfully!")

✅ Libraries installed and imported successfully!


Step 2: Load Dataset (SQuAD v1.1)  
I loaded the Stanford Question Answering Dataset (SQuAD v1.1), which contains passages, questions, and answers.  
The dataset is split into training and validation sets.

In [ ]:
dataset = load_dataset("squad")
print("✅ Dataset loaded successfully!")
print(dataset)

✅ Dataset loaded successfully!
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


Step 3: Load Pre-trained Model and Tokenizer  
I used the pre-trained DistilBERT model fine-tuned on SQuAD.  
The tokenizer converts text into numerical tokens, and the model predicts answer spans.

In [ ]:
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

print("✅ Model and tokenizer loaded successfully!")

✅ Model and tokenizer loaded successfully!


Step 4: Create Question Answering Pipeline  
I created a Hugging Face pipeline for question answering.  
By providing a question and a context passage, the model extracts the correct answer.

In [ ]:
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

context = dataset["validation"][0]["context"]
question = dataset["validation"][0]["question"]

result = qa_pipeline(question=question, context=context)
print("Question:", question)
print("Answer:", result["answer"])

Device set to use cpu


Question: Which NFL team represented the AFC at Super Bowl 50?
Answer: Denver Broncos


Step 5: Evaluate the Model  
I evaluated the model on a subset of 200 validation samples.  
The evaluation metrics include Exact Match (EM) and F1 Score, which measure how well the predicted answers match the ground truth.

In [ ]:
metric = evaluate.load("squad")

n_samples = 200
predictions = []
references = []

for i in range(n_samples):
    context = dataset["validation"][i]["context"]
    question = dataset["validation"][i]["question"]
    answers = dataset["validation"][i]["answers"]

    result = qa_pipeline(question=question, context=context)

    predictions.append({
        "id": dataset["validation"][i]["id"],
        "prediction_text": result["answer"]
    })
    references.append({
        "id": dataset["validation"][i]["id"],
        "answers": answers
    })

scores = metric.compute(predictions=predictions, references=references)
print("✅ Evaluation Results:", scores)



✅ Evaluation Results: {'exact_match': 85.0, 'f1': 89.3917748917749}


Step 6 (Bonus): Try Another Model  
As a bonus step, I tested another model (RoBERTa fine-tuned on SQuAD).  
This allowed me to compare the performance of different transformer-based models.

In [ ]:
alt_model = "deepset/roberta-base-squad2"
qa_pipeline_alt = pipeline("question-answering", model=alt_model, tokenizer=alt_model)

sample_q = "Who was the first president of the United States?"
sample_c = "George Washington was the first president of the United States and served from 1789 to 1797."

print("\nBonus Test with RoBERTa:")
print("Question:", sample_q)
print("Answer:", qa_pipeline_alt(question=sample_q, context=sample_c)["answer"])

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu



Bonus Test with RoBERTa:
Question: Who was the first president of the United States?
Answer: George Washington
